In [2]:
import sys

!{sys.executable} -m pip install "sagemaker>=2.99.0"


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import json

# Opening JSON file
with open('pipeline.json', 'r') as openfile:

	# Reading from json file
	json_object = json.load(openfile)

#print(json_object)
#print(type(json_object))

In [4]:
import os

import boto3
import sagemaker

from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)

from sagemaker import Model
from sagemaker.xgboost import XGBoostPredictor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep, CacheConfig, TuningStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep

from sagemaker.workflow.functions import Join, JsonGet
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.tuner import (
    ContinuousParameter,
    HyperparameterTuner,
    WarmStartConfig,
    WarmStartTypes,
)

In [5]:
# Create the SageMaker Session

region = sagemaker.Session().boto_region_name
sm_client = boto3.client("sagemaker")
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sm_client)

# Create a Pipeline Session
pipeline_session = PipelineSession()

In [9]:
# Define variables and parameters needed for the Pipeline steps

role = sagemaker.get_execution_role()
#role=json_object['Steps'][0]['Arguments']['RoleArn']
default_bucket = sagemaker_session.default_bucket()
base_job_prefix = "tuning-step-example"
model_package_group_name = "tuning-job-model-packages"

processing_instance_count = ParameterInteger(name=json_object['Parameters'][0]['Name'], default_value=json_object['Parameters'][0]['DefaultValue'])
training_instance_type = ParameterString(name=json_object['Parameters'][1]['Name'], default_value=json_object['Parameters'][1]['DefaultValue'])

input_data = ParameterString(
    name=json_object['Parameters'][2]['Name'],
    default_value=json_object['Parameters'][2]['DefaultValue'],
)
model_approval_status = ParameterString(
    name=json_object['Parameters'][4]['Name'], default_value=json_object['Parameters'][4]['DefaultValue']
)

local_path = "data/abalone-dataset-batch"
#base_uri = f"s3://{default_bucket}/abalone"
# batch_data_uri = sagemaker.s3.S3Uploader.upload(
#     local_path=local_path,
#     desired_s3_uri=base_uri,
# )
batch_data_uri=json_object['Parameters'][3]['DefaultValue']
print(batch_data_uri)
batch_data = ParameterString(
    name=json_object['Parameters'][3]['Name'],
    default_value=batch_data_uri,
)

# Cache Pipeline steps to reduce execution time on subsequent executions
cache_config = CacheConfig(enable_caching=True, expire_after="30d")

s3://sagemaker-us-east-1-368703438116/abalone/abalone-dataset-batch


# Data Preparation



In [12]:
# Process the training data step using a python script.
# Split the training data set into train, test, and validation datasets
# When defining the ProcessingOutput destination as a dynamic value using the
# Pipeline Execution ID, caching will not be in effect as each time the step runs,
# the step definition changes resulting in new execution. If caching is required,
# the ProcessingOutput definition should be status
json_prep=json_object['Steps'][0]

sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    instance_type=json_prep['Arguments']['ProcessingResources']['ClusterConfig']['InstanceType'],
    instance_count=processing_instance_count,
    base_job_name=f"{base_job_prefix}/sklearn-abalone-preprocess",
    sagemaker_session=pipeline_session,
    role=role,
)

processor_run_args = sklearn_processor.run(
    outputs=[
        ProcessingOutput(
            output_name="train",
            source=json_prep['Arguments']['ProcessingOutputConfig']['Outputs'][0]['S3Output']['LocalPath'],
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    default_bucket,
                    base_job_prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "PreprocessAbaloneDataForHPO",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="validation",
            source=json_prep['Arguments']['ProcessingOutputConfig']['Outputs'][1]['S3Output']['LocalPath'],
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    default_bucket,
                    base_job_prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "PreprocessAbaloneDataForHPO",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="test",
            source=json_prep['Arguments']['ProcessingOutputConfig']['Outputs'][2]['S3Output']['LocalPath'],
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    default_bucket,
                    base_job_prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "PreprocessAbaloneDataForHPO",
                ],
            ),
        ),
    ],
    code="code/preprocess.py",
    arguments=["--input-data", input_data],
)

step_process = ProcessingStep(
    name=json_prep['Name'],
    step_args=processor_run_args,
)

/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:198: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,



Job Name:  tuning-step-example/sklearn-abalone-pre-2022-08-02-02-04-12-767
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-368703438116/tuning-step-example/sklearn-abalone-pre-2022-08-02-02-04-12-767/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train', 'AppManaged': False, 'S3Output': {'S3Uri': Join(on='/', values=['s3:/', 'sagemaker-us-east-1-368703438116', 'tuning-step-example', <sagemaker.workflow.execution_variables.ExecutionVariable object at 0x7fba6a400990>, 'PreprocessAbaloneDataForHPO']), 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation', 'AppManaged': False, 'S3Output': {'S3Uri': Join(on='/', values=['s3:/', 'sagemaker-us-east-1-368703438116', 'tuning-step-example', <sagemaker.workflow.execution_vari

# Hyperparameter Tuning



In [13]:
json_tuning=json_object['Steps'][1]

In [14]:
staticHyper=json_tuning['Arguments']['TrainingJobDefinition']['StaticHyperParameters']

In [15]:
json_hyperp_ranges=json_tuning['Arguments']['HyperParameterTuningJobConfig']['ParameterRanges']['ContinuousParameterRanges']

In [16]:
# Define the output path for the model artifacts from the Hyperparameter Tuning Job
model_path = json_tuning['Arguments']['TrainingJobDefinition']['OutputDataConfig']['S3OutputPath']

image_uri = json_tuning['Arguments']['TrainingJobDefinition']['AlgorithmSpecification']['TrainingImage']

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=json_tuning['Arguments']['TrainingJobDefinition']['ResourceConfig']['InstanceCount'],
    output_path=model_path,
    base_job_name=f"{base_job_prefix}/abalone-train",
    sagemaker_session=pipeline_session,
    role=role,
)

xgb_train.set_hyperparameters(
    eval_metric=staticHyper['eval_metric'],
    objective=staticHyper['objective'],  # Define the object metric for the training job
    num_round=staticHyper['num_round'],
    max_depth=staticHyper['max_depth'],
    eta=staticHyper['eta'],
    gamma=staticHyper['gamma'],
    min_child_weight=staticHyper['min_child_weight'],
    subsample=staticHyper['subsample'],
    silent=staticHyper['silent'],
)

objective_metric_name = json_tuning['Arguments']['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

hyperparameter_ranges = {
    json_hyperp_ranges[0]['Name']: ContinuousParameter(json_hyperp_ranges[0]['MinValue'], json_hyperp_ranges[0]['MaxValue'], scaling_type=json_hyperp_ranges[0]['ScalingType']),
    json_hyperp_ranges[1]['Name']: ContinuousParameter(json_hyperp_ranges[1]['MinValue'], json_hyperp_ranges[1]['MaxValue'], scaling_type=json_hyperp_ranges[1]['ScalingType']),
}

tuner_log = HyperparameterTuner(
    xgb_train,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=json_tuning['Arguments']['HyperParameterTuningJobConfig']['ResourceLimits']['MaxNumberOfTrainingJobs'],
    max_parallel_jobs=json_tuning['Arguments']['HyperParameterTuningJobConfig']['ResourceLimits']['MaxParallelTrainingJobs'],
    strategy=json_tuning['Arguments']['HyperParameterTuningJobConfig']['Strategy'],
    objective_type=json_tuning['Arguments']['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'],
)

hpo_args = tuner_log.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

step_tuning = TuningStep(
    name=json_tuning['Name'],
    step_args=hpo_args,
    cache_config=cache_config,
)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


# Creating best models



In this example, the two best models from the TuningStep are added to the same model package group in the Model Registry as v0 and v1.

You use the `get_top_model_s3_uri` method of the TuningStep class to get the model artifact from one of the top performing model versions

In [17]:
json_create1=json_object["Steps"][3]['Arguments']['IfSteps'][0]
json_create2=json_object["Steps"][3]['Arguments']['IfSteps'][1]

In [18]:
# Creating 2 SageMaker Models
model_bucket_key = json_create1['Arguments']['PrimaryContainer']['ModelDataUrl']['Std:Join']['Values'][1]

best_model = Model(
    image_uri=image_uri,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_bucket_key),
    predictor_cls=XGBoostPredictor,
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_first = ModelStep(
    name="CreateBestModel",
    step_args=best_model.create(instance_type="ml.m5.xlarge"),
)

second_best_model = Model(
    image_uri=image_uri,
    model_data=step_tuning.get_top_model_s3_uri(top_k=1, s3_bucket=model_bucket_key),
    predictor_cls=XGBoostPredictor,
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_second = ModelStep(
    name="CreateSecondBestModel",
    step_args=second_best_model.create(instance_type="ml.m5.xlarge"),
)

# Evaluate the model



In [20]:
json_eval=json_object['Steps'][2]

In [22]:
# A ProcessingStep is used to evaluate the performance of a selected model from the HPO step. In this case, the top performing model
# is evaluated. Based on the results of the evaluation, the model is registered into the Model Registry using a ConditionStep.

script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=[json_eval['Arguments']['AppSpecification']['ContainerEntrypoint'][0]],
    instance_type=json_object['Steps'][2]['Arguments']['ProcessingResources']['ClusterConfig']['InstanceType'],
    instance_count=json_object['Steps'][2]['Arguments']['ProcessingResources']['ClusterConfig']['InstanceCount'],
    base_job_name=f"{base_job_prefix}/script-tuning-step-eval",
    sagemaker_session=pipeline_session,
    role=role,
)

evaluation_report = PropertyFile(
    name=json_object['Steps'][2]['PropertyFiles'][0]['PropertyFileName'],
    output_name=json_object['Steps'][2]['PropertyFiles'][0]['OutputName'],
    path=json_object['Steps'][2]['PropertyFiles'][0]['FilePath'],
)

processor_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_bucket_key),
            destination=json_eval['Arguments']['ProcessingInputs'][0]['S3Input']['LocalPath'],
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination=json_eval['Arguments']['ProcessingInputs'][1]['S3Input']['LocalPath'],
        ),
    ],
    outputs=[
        ProcessingOutput(output_name=json_eval['Arguments']['ProcessingOutputConfig']['Outputs'][0]['OutputName'], source=json_eval['Arguments']['ProcessingOutputConfig']['Outputs'][0]['S3Output']['LocalPath']),
    ],
    code="code/evaluate.py",
)

# This can be extended to evaluate multiple models from the HPO step
step_eval = ProcessingStep(
    name=json_eval['Name'],
    step_args=processor_args,
    property_files=[evaluation_report],
    cache_config=cache_config,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)


Job Name:  tuning-step-example/script-tuning-step--2022-08-02-02-04-56-327
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': Join(on='/', values=['s3:/', 'sagemaker-us-east-1-368703438116/tuning-step-example/AbaloneTrain', <sagemaker.workflow.properties.Properties object at 0x7fba6893e890>, 'output/model.tar.gz']), 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': <sagemaker.workflow.properties.Properties object at 0x7fba69aac8d0>, 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-368703438116/tuning-step-example/script-tuning-step--2022-08-02-02-04-56-327/input/code/evaluate.p

# Register Model

In [23]:
json_register=json_object['Steps'][3]['Arguments']['IfSteps'][2]['Arguments']['InferenceSpecification']

In [24]:
# Register the model in the Model Registry
# Multiple models can be registered into the Model Registry using multiple ModelSteps. These models can either be added to the
# same model package group as different versions within the group or the models can be added to different model package groups.

register_args = best_model.register(
    content_types=json_register['SupportedContentTypes'],
    response_types=json_register['SupportedResponseMIMETypes'],
    inference_instances=json_register['SupportedRealtimeInferenceInstanceTypes'],
    transform_instances=json_register['SupportedTransformInstanceTypes'],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)


step_register_best = ModelStep(name="RegisterBestAbaloneModel", step_args=register_args)

# Setting up Lambda



In [29]:

lambda_role = "arn:aws:iam::368703438116:role/lambda-deployment-role"

In [30]:
json_lambda=json_object['Steps'][3]['Arguments']['IfSteps'][3]

In [31]:
json_lambda['Arguments']

{'model_name': {'Get': 'Steps.CreateBestModel-CreateModel.ModelName'},
 'endpoint_config_name': 'demo-lambda-deploy-endpoint-config-08-01-04-21-10',
 'endpoint_name': 'demo-lambda-deploy-endpoint-08-01-04-21-10'}

In [32]:
# Custom Lambda Step
import time
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)


#current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
#model_name = "demo-lambda-model" + current_time

endpoint_config_name = json_lambda['Arguments']['endpoint_config_name']
endpoint_name = json_lambda['Arguments']['endpoint_name']

function_name = "sagemaker-lambda-step-endpoint-deploy-" + "08-01-04-21-10"

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="code/lambda_helper.py",
    handler="lambda_helper.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    name="LambdaStep",
    lambda_func=func,
    inputs={
        "model_name": step_create_first.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

# Transform Step 



In [33]:
json_transform=json_object['Steps'][3]['Arguments']['IfSteps'][4]

In [34]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name=step_create_first.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=json_transform['Arguments']['TransformResources']['InstanceCount'],
    output_path=json_transform['Arguments']['TransformOutput']['S3OutputPath'],
)

Pass in the transformer instance and the `TransformInput` with the `batch_data` pipeline parameter defined earlier.

In [35]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="AbaloneTransform", transformer=transformer, inputs=TransformInput(data=batch_data)
)

# Fail Step

In [36]:
json_fail=json_object['Steps'][3]['Arguments']['ElseSteps'][0]

In [37]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join
mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)
step_fail = FailStep(
    name=json_fail['Name'],
    error_message=Join(on=json_fail['Arguments']['ErrorMessage']['Std:Join']['On'], 
                       values=json_fail['Arguments']['ErrorMessage']['Std:Join']['Values']),
)

# Condition Step

In [38]:
json_cond=json_object['Steps'][3]['Arguments']['Conditions'][0]

In [39]:
# condition step for evaluating model quality and branching execution

cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path=json_cond['LeftValue']['Std:JsonGet']['Path'],
    ),
    right=json_cond['RightValue'],
)
step_cond = ConditionStep(
    name=json_object['Steps'][3]['Name'],
    conditions=[cond_lte],
    if_steps=[
        step_create_first,
        step_create_second,
        step_register_best, 
        step_deploy_lambda,
        step_transform,
    ],
    else_steps=[step_fail],
)

# Pipeline

In [40]:
pipeline = Pipeline(
    name="mlops-pipeline",
    parameters=[
        processing_instance_count,
        training_instance_type,
        input_data,
         batch_data,
        model_approval_status,
    ],
    steps=[
        step_process,
        step_tuning,
        step_eval,
        step_cond,
    ],
    sagemaker_session=pipeline_session,
)

#### Execute the Pipeline

In [41]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'InputDataUrl',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-sample-files/datasets/tabular/uci_abalone/abalone.csv'},
  {'Name': 'BatchData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-368703438116/abalone/abalone-dataset-batch'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'PreprocessAbaloneDataForHPO',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'Volume

In [42]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:368703438116:pipeline/mlops-pipeline',
 'ResponseMetadata': {'RequestId': 'f311f754-d26b-4fd1-84ff-7c5876b67817',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f311f754-d26b-4fd1-84ff-7c5876b67817',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Tue, 02 Aug 2022 02:05:34 GMT'},
  'RetryAttempts': 0}}

In [43]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:368703438116:pipeline/mlops-pipeline/execution/gsbhtt26s993', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x7fba68fdec50>)

#### Cleaning up resources

Users are responsible for cleaning up resources created when running this notebook. Specify the ModelName, ModelPackageName, and ModelPackageGroupName that need to be deleted. The model names are generated by the CreateModel step of the Pipeline and the property values are available only in the Pipeline context. To delete the models created by this pipeline, navigate to the Model Registry and Console to find the models to delete.

In [137]:
# Get the model name from the EndpointCofig. The CreateModelStep properties are not available
# outside the Pipeline execution context so `step_create_model.properties.ModelName`
# cannot be used while deleting the model.

model_name = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)[
    "ProductionVariants"
][0]["ModelName"]

# Delete the Model
sm_client.delete_model(ModelName=model_name)

# Delete the EndpointConfig
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete the Endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

# Delete the Lambda function
func.delete()

# Delete the Pipeline
sm_client.delete_pipeline(PipelineName="mlops-pipeline")

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:368703438116:pipeline/mlops-pipeline',
 'ResponseMetadata': {'RequestId': 'c613f531-0e86-431a-a565-f15507a4ecb6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c613f531-0e86-431a-a565-f15507a4ecb6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Mon, 01 Aug 2022 08:50:40 GMT'},
  'RetryAttempts': 0}}